<a href="https://colab.research.google.com/github/Esandu-Meth-Obadaarachchi/HCHO-level-Time-series-forecasting/blob/main/DataEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pyspark
import pyspark.sql  as pyspark_sql
import pyspark.sql.types as pyspark_types
import pyspark.sql.functions  as pyspark_functions
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, regexp_replace, when
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = pyspark_sql.SparkSession.builder.getOrCreate()


In [ ]:
# Define column names and data types
schema = StructType([
    StructField("HCHO reading", DoubleType(), True),
    StructField("Location", StringType(), True),
    StructField("Current Date", StringType(), True),
    StructField("Next Date", StringType(), True)
])

# Load the data for 'Colombo Proper', 'Deniyaya, Matara', 'Nuwara Eliya Proper' with specified column names
col_mat_nuw_df = spark.read.csv("/content/drive/MyDrive/data engineering CW/col_mat_nuw_output.csv", header=False, schema=schema)

# Load the data for 'Bibile, Monaragala', 'Kurunegala Proper', 'Jaffna Proper' with specified column names
mon_kur_jaf_df = spark.read.csv("/content/drive/MyDrive/data engineering CW/mon_kur_jaf_output.csv", header=False, schema=schema)

# Load the data for 'Kandy Proper' with specified column names
kan_df = spark.read.csv("/content/drive/MyDrive/data engineering CW/kan_output.csv", header=False, schema=schema)


In [ ]:
col_mat_nuw_df.show()